In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map  

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

In [4]:
roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [5]:
image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20181102').clip(roi)
Map.addLayer(AWEIsh(image), visParams, 'image')

In [6]:
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

awei_water = AWEIsh(image).select('AWEI').gt(0)
Map.addLayer(awei_water.selfMask(),{'palette':['blue']}, 'awei_water')
# Map.addLayer(awei_water,{'palette':['white','blue'],'min':0,'max':1}, 'awei_water')

# 计算在75%处，AWEI的值

In [7]:
def percentile(image):
    a = image.reduceRegion(**{
        'reducer': ee.Reducer.percentile([75]), 
        'geometry': roi, 
        'scale': 30, 
        'bestEffort': True
    })
    return image.set(a)
a = AWEIsh(image).select('AWEI')
percent_75 = percentile(a).get('AWEI')
print(percent_75.getInfo())

-1922.9553450346796


In [8]:
a = AWEIsh(image).select('AWEI')
intervalMean = a.reduceRegion(**{
    'reducer': ee.Reducer.intervalMean(75,100), 
    'geometry': roi, 
    'scale': 30, 
    'bestEffort': True
})
print(intervalMean.getInfo())
mask = a.gt(ee.Image.constant(intervalMean.get('AWEI')))

Map.addLayer(mask.selfMask(),{'palette':['2ca25f']}, 'mask')

{'AWEI': 431.8262579924745}


In [ ]:
a = AWEIsh(image).select('AWEI')
percentile = a.reduceRegion(**{
    'reducer': ee.Reducer.percentile([75]), 
    'geometry': roi, 
    'scale': 30, 
    'bestEffort': True
})
mask = a.gt(ee.Image.constant(percentile.get('AWEI')))
Map.addLayer(mask.selfMask(),{'palette':['2ca25f']}, 'mask')

In [ ]:
a = ee.List([1,2,3,4,5,6,7,8,9,10])
percentile = a.reduce(ee.Reducer.percentile([0,10,20,30,40,50,60,70,80,90,100]))
intervalMean = a.reduce(ee.Reducer.intervalMean(80,100))
print(percentile.getInfo())
print(intervalMean.getInfo())

In [ ]:
awei_78_100 = AWEIsh(image).select('AWEI').gt(557.2084534892857)
Map.addLayer(awei_78_100.selfMask(),{'palette':['2ca25f']}, 'awei_78_100')

awei_80_100 = AWEIsh(image).select('AWEI').gt(930.8355484041551)
Map.addLayer(awei_80_100.selfMask(),{'palette':['red']}, 'awei_80_100')

# 频率滤波

In [ ]:
kernel = ee.Kernel.square(1,'pixels',False)
mode = ee.Reducer.mode(2,1)
reducer_water = awei_water.reduceNeighborhood(mode,kernel)
print(reducer_water.bandNames().getInfo())
# Map.addLayer(reducer_water,{'palette':['white','red'],'min':0,'max':1}, 'reducer_water')
Map.addLayer(reducer_water.selfMask(),{'palette':['red']}, 'reducer_water')

In [ ]:
focal_mode = awei_water.focal_mode(1,'square','pixels')
print(focal_mode.bandNames().getInfo())
Map.addLayer(focal_mode,{'palette':['white','red'],'min':0,'max':1}, 'focal_mode')

geemap.ee_export_image(ee_object=focal_mode,filename='focal_mode.tif',scale=30,region=roi.geometry(),file_per_band=False,)

In [ ]:
# geemap.ee_export_image(ee_object=awei_water,filename='awei_water.tif',scale=30,region=roi.geometry(),file_per_band=False,)

In [ ]:
# for i in range(5,11,1):
#     reducer_water = awei_water.reduceNeighborhood(ee.Reducer.mode(2,1),ee.Kernel.square(i,'pixels',False))
#     geemap.ee_export_image(ee_object=reducer_water,filename='reducer_water{}.tif'.format(i),scale=30,region=roi.geometry(),file_per_band=False,)

In [ ]:
# 武汉
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['CDCDCD','0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(reducer_water, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(wuhan)', fontsize=20)